# 1. Introduction

In [ ]:
""" I am attempting to make a model out of this research paper on 
MULTI-VIEW GAIT RECOGNITION USING 3D CONVOLUTIONAL NEURAL NETWORKS. it's my first time, that I am
building a model from scratch. I am excited to use Keras for this purpose."""

"""The prerequisites:
1. Keras, Tensorflow, Python, Numpy, Matplotlib, pandas
2. Reading and analysing the research papers
3. Knowledge of 3D convolution neural networks
4. CASIA_B Dataset and it's ordering and structuring inside the given zip file
5. A lot of faith in one's self and willing to take risk with one's time!

So if you have done this, cheers! Let's go!"""

"""Important points:
1. Referring to https://towardsdatascience.com/
step-by-step-implementation-3d-convolutional-neural-network-in-keras-12efbdd7b130

2. I am taking 20 images out of the 6 walking sequences from each of the 11 viewing angles in 
normal/cooperative conditions for 124 total subjects

3. I have taken 20 more images out of the same above given categories for my testing data
"""

"""I will be noting the experiment count here: """
count = 9
print("This is experiment number: ",count)

# 2. Importing Libraries 

In [1]:
import os
import numpy as np
import PIL
import cv2
import time
import tensorflow as tf
from PIL import Image
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv3D
from keras.layers import AveragePooling2D, MaxPooling3D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.utils import to_categorical
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline


# 3. Data Preprocessing

In [2]:
# structuring the data
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
time_step=0
frames=[]
filepath=r"D:\dataScience\machine-learning-online-2018-master\Datasets\GaitDatasetB-silh"
condition="nm-01"
start=time.time()
for person in os.listdir(filepath):
    mainpath=os.path.join(filepath,person,condition)
    for view in os.listdir(mainpath):
        total_path=os.path.join(mainpath,view)
        count=0
        for i,image in enumerate(os.listdir(total_path)):
            orig=Image.open(os.path.join(total_path,image))
            orig=orig.resize((128,192))
            img=Image.new("RGB",orig.size)
            img.paste(orig)
            if time_step==16:
                if i<18: # can be any random number less than 32 actually as it's just a check!
                    X_train.append(np.asarray(frames))
                    Y_train.append(int(person.strip())-1)
                
                else:
                    X_test.append(np.asarray(frames))
                    Y_test.append(int(person.strip())-1)
                time_step=0
                frames=[]
                count+=1
            
            if count==2:
                break
                
            frames.append(np.asarray(img))
            time_step+=1
        
end=time.time()

print("the time taken to parse and restructure the dataset is : {0} seconds".format(end-start))


        

the time taken to parse and restructure the dataset is : 61.095455169677734 seconds


In [3]:
#converting to numpy arrays
X_train=np.array(X_train)
Y_train=np.array(Y_train)
X_test=np.array(X_test)
Y_test=np.array(Y_test)


In [4]:
idx = np.random.permutation(len(X_train))
X_train,Y_train = X_train[idx], Y_train[idx]
print(Y_train[:10])

[ 12  62  61  74  36  68  58   4 112 116]


In [5]:
idx = np.random.permutation(len(X_test))
X_test,Y_test = X_test[idx], Y_test[idx]
print(Y_test[:10])

[ 64  28  93  86  59  25 110  54  95  36]


In [6]:
print(X_train[0])

[[[[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  ...

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]]


 [[[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  ...

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]]


 [[[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0

In [7]:
#X_train = rgb_data_transform(X_train)
#X_test = rgb_data_transform(X_test)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

# Convert target vectors to categorical targets
targets_train = to_categorical(Y_train).astype(np.integer)
targets_test = to_categorical(Y_test).astype(np.integer)
#dictating the sample shape

(1353, 16, 192, 128, 3)
(1353,)
(1346, 16, 192, 128, 3)
(1346,)


In [8]:
print(targets_train.shape)
print(targets_test.shape)

(1353, 124)
(1346, 124)


# 4. Creating and Training the model

In [9]:
# reshaping X_train a little bit
X_train=np.swapaxes(X_train,1,2)
X_train=np.swapaxes(X_train,2,3)

In [10]:
print(X_train.shape)

(1353, 192, 128, 16, 3)


In [ ]:
# Create the model
model = Sequential()
#layer1
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same', data_format='channels_last', kernel_initializer='he_uniform', 
                 input_shape=(192,128,16,3))) 
model.add(MaxPooling3D(pool_size=(2, 2, 1), data_format='channels_last'))
#layer2
model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same', 
                 data_format='channels_last', kernel_initializer='he_uniform' ))
model.add(MaxPooling3D(pool_size=(2, 2, 2), data_format='channels_last'))
#layer3
model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same', data_format='channels_last', 
                 kernel_initializer='he_uniform' ))
model.add(MaxPooling3D(pool_size=(2, 2, 1), data_format='channels_last'))
#layer4
model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding='same', data_format='channels_last', 
                 kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), data_format='channels_last'))
#layer5
model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding='same', data_format='channels_last', 
                 kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), data_format='channels_last'))
#layer6
model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same', data_format='channels_last', 
                 kernel_initializer='he_uniform'))
#layer7
model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same', data_format='channels_last', 
                 kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), data_format='channels_last'))
#flattening
model.add(Flatten())
#FC layer 1
model.add(Dense(4096, activation='relu', kernel_initializer='he_uniform', 
                kernel_regularizer=tf.keras.regularizers.l2(0.0005)))
model.add(Dropout(0.5))
#FC Layer 2
model.add(Dense(4096, activation='relu', kernel_initializer='he_uniform',  
                kernel_regularizer=tf.keras.regularizers.l2(0.0005)))
model.add(Dropout(0.5))
# making a softmax function
model.add(Dense(124, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.summary()
# Fit data to model
history = model.fit(X_train, targets_train,
            batch_size=16,
            epochs=40,
            verbose=1,
            validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 192, 128, 16, 64)  5248      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 96, 64, 16, 64)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 96, 64, 16, 128)   221312    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 48, 32, 8, 128)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 48, 32, 8, 128)    442496    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 24, 16, 8, 128)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 16, 8, 256)    8

# 5. Testing the model

In [ ]:
preds = model.evaluate(X_test,targets_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))